**# The overall algorithm:**


    **1. Create embeddings of all cells.**
    **2. Given a query, process the query to not have tonality and be specific.**
    **3. Find the most similar to the query cells.**
    **4. Filter excel with some additions on how to recover important cells.**
    **5. Feed to llm a much smaller table with all noisy information omitted.**


Import dependencies

In [1]:
import os
import requests
import json
import pandas as pd
import sys
import openpyxl
import random
import itertools
import pandas as pd
from pinecone import Pinecone
from openpyxl import Workbook, load_workbook
from openai import OpenAI
import concurrent.futures
from data_formatting import *
from agents import *


# !pip3 install python-dotenv
from dotenv import load_dotenv

/Users/artemsaraiev/stud/sheetseeker/venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# Get the current working directory
current_dir = os.getcwd()

# Construct the path to the .env file relative to the current directory
env_path = os.path.join(current_dir, '..', '.env')

# Load environment variables from .env file
load_dotenv(env_path)

True

In [3]:
SECRET_KEY = os.environ.get('SECRET_KEY')
EMBED_API_KEY = os.environ.get('EMBED_API_KEY')
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENROUTER_API_KEY = os.environ.get('OPENROUTER_API_KEY')
EMBED_MODEL = os.environ.get('EMBED_MODEL')
BASE_MODEL = os.environ.get('BASE_MODEL')
INDEX_NAME = os.environ.get('INDEX_NAME')

In [4]:
pc = Pinecone(api_key=PINECONE_API_KEY)
INDEX = pc.Index(INDEX_NAME)
INDEX_DIMENSION = 1536

Each namespace in the index stores vectors; we will use one namespace per table. 
You can find more about namespaces at [pinecone.io](https://docs.pinecone.io/guides/getting-started/quickstart).

Before pushing vectors to a namespace, clear the namespace so it has consists only of specified vectors.

In [5]:
def clear_namespace(namespace, index = INDEX):
    # just add a single vector to the namespace, because 
    # deleting non-existing namespace will raise an error
    index.upsert(
    vectors=[
        {"id": "A", "values": [0.1]*INDEX_DIMENSION},
    ],
    namespace=namespace,
    )
    index.delete(namespace = namespace, delete_all = True)

Create embeddings concurrently.

In [6]:
def create_embeddings(terms, embed_model=EMBED_MODEL):
    temp_client = OpenAI(api_key=EMBED_API_KEY)
    def get_embedding(obj):
        response = temp_client.embeddings.create(
            input=obj['text_to_embed'],
            model=embed_model,
        )
        embedding_vector = response.data[0].embedding
        obj['values'] = embedding_vector
        return obj

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_embeddings = [executor.submit(get_embedding, term) for term in terms]
        results = [future.result() for future in concurrent.futures.as_completed(future_embeddings)]
    return results

Upload (upsert) financial term objects to the pinecone in chunks

In [7]:
def upsert_chunks(terms, namespace, index=INDEX):
    embeds = create_embeddings(terms)
    new_embeds = []
    for emb in embeds:
        new_embeds.append({"id": emb["id"], 
                           "values": emb["values"], 
                           "metadata": {
                                "sheet": emb["sheet"],
                                "row": emb["row"],
                                "col": emb["col"],
                                "window_width": emb["window_width"],
                                "window_height": emb["window_height"],
                                "type": emb["type"]
                                }   
                        })

    chunks = [new_embeds[i:i + 100] for i in range(0, len(new_embeds), 100)]

    for chunk in chunks:
        index.upsert(vectors=chunk, namespace=namespace)

Same but for `queries`, which have different representation.

In [8]:
def upsert_chunks_query(queries, namespace, index=INDEX):
    embeds = create_embeddings(queries)
    new_embeds = []
    for emb in embeds:
        new_embeds.append({"id": emb["id"],
                           "values": emb["values"],   
                        })

    chunks = [new_embeds[i:i + 100] for i in range(0, len(new_embeds), 100)]

    for chunk in chunks:
        index.upsert(vectors=chunk, namespace=namespace)

Generates financial term embeddings from the given table.

In [9]:
def generate_fin_term_embeds(file_path):
    def get_fin_terms_cell():
        # Open the workbook with data_only=True to get the values instead of formulas
        workbook = openpyxl.load_workbook(file_path, data_only=True)
        terms = []
        for sheet_name in workbook.sheetnames:
            sheet = workbook[sheet_name]
            
            # Iterate over all cells in the current sheet
            for row in sheet.iter_rows():
                for term in row:
                    common_data = {
                        "sheet": sheet_name,
                        "row": term.row,
                        "col": term.column,
                        "window_width": 0,
                        "window_height": 0,
                    }
                    if term.value is None or isinstance(term.value, str) and not term.value.strip():
                        continue
                    text_to_embed = str(term.value)
                    # print(f'{text_to_embed = }')
                    if not text_to_embed.isnumeric():
                        common_data.update({"text_to_embed": text_to_embed})
                        common_data.update({"type": "term"})
                    elif text_to_embed.isnumeric():
                        # Find the first text cell to the left
                        left_cell = term
                        left_text = ""
                        while left_cell.column > 1:
                            left_cell = sheet.cell(row=left_cell.row, column=left_cell.column - 1)
                            if isinstance(left_cell.value, str) and not left_cell.value.isnumeric():
                                left_text = left_cell.value
                                break

                        up_cell = term
                        up_text = ""
                        while up_cell.row > 1:
                            up_cell = sheet.cell(row=up_cell.row - 1, column=up_cell.column)
                            if isinstance(up_cell.value, str):
                                up_text = up_cell.value
                                break
                        text_to_embed = left_text + " " + up_text
                        # print(f'Combined text to embed: {text_to_embed}')
                        common_data.update({"text_to_embed": text_to_embed})
                        common_data.update({"type": "value"})
                    terms.append(common_data)
        return terms
    return get_fin_terms_cell()

In [10]:
def generate_embeddings(file_path):
    new_namespace = file_path
    clear_namespace(new_namespace, INDEX)

    fin_term_embeds = generate_fin_term_embeds(file_path)  # Assume this function is defined elsewhere
    # sliding_window_embeds = generate_sliding_window_emeds(file_path)  # Assume this function is defined elsewhere
    combined_terms = fin_term_embeds #+ sliding_window_embeds
    chunk_id = 1
    for term in combined_terms:
        term['id'] = str(chunk_id)
        chunk_id +=1
    upsert_chunks(combined_terms, file_path)

Gets top-k similar vectors for a given query. For larger tables top-k should be scaled accordingly.

In [11]:
def get_similarities(query, namespace, index = INDEX, top_k=500, filter = None):
    temp_client = OpenAI(
    api_key=os.environ.get('EMBED_API_KEY'),
    )

    description_response = temp_client.embeddings.create(
            input=query,
            model=EMBED_MODEL,
        )
    query_embed_vec = description_response.data[0].embedding
    if filter is None:
        query = index.query(
            namespace=namespace,
            vector=query_embed_vec,
            top_k=top_k,
            include_metadata=True
        )
    else:
        query = index.query(
            namespace=namespace,
            vector=query_embed_vec,
            filter = filter,
            top_k=top_k,
            include_metadata=True
        )
    return query

In [12]:
def get_formatted_quiries(user_query):
    filtered_query = FormatQuestionCall(user_query)
    filtered_query.respond()
    queries_formatted =  ({"text_to_embed": query, "id": str(ix+1)} for ix, query in enumerate(filtered_query.queries))
    print(f'{filtered_query.queries = }')
    return queries_formatted, filtered_query

In [13]:
def embed_quiries(queries_formatted):

    new_namespace = "QUERIES"
    clear_namespace(new_namespace, INDEX)
    upsert_chunks_query(queries_formatted, "QUERIES")

In [14]:
def answer_single_query(query, table_path):
        similarities = get_similarities(query, table_path, INDEX)

        # create filtered table
        # cell_set = create_similar_cell_set(similarities, threshold=0.2)
        filtered_path = f"filtered_table.xlsx"
        print(f'filtered excel for {table_path} is being created at {filtered_path}')

        create_filtered_meta_excel(table_path, filtered_path, similarities)
        # create_filtered_excel(table_path, filtered_path, cell_set)

        answer_call = AnswerCall(query)
        answer_call.load_table(filtered_path)
        answer_call.respond()

        query_result = answer_call.highlighted_json
        return query_result


In [15]:
def process_user_query(user_query, table_path):
    queries_formatted, filtered_query = get_formatted_quiries(user_query)
    embed_quiries(queries_formatted)

    query_answers_list = []

    for query in filtered_query.queries:
        query_result = answer_single_query(query, table_path)
        query_answers_list.append(json_to_string(query_result))
       
    return query_answers_list

Let's now load a table, and create embeddings.

In [16]:
filepath = 'tutorial_data/sample_input.xlsx'
generate_embeddings(filepath)

Embeddings are now generated and stored in `filepath` namespace. 

Let's ask anything about this table.

In [17]:
user_query = "Revenue"

In [18]:
answer_embeddings = process_user_query(user_query, filepath) 

filtered_query.queries = ['Total Revenue']
filtered excel for tutorial_data/sample_input.xlsx is being created at filtered_table.xlsx
66


You can see how the most valuable to the "Revenue" request cells were left in the table.

Answer was prompted to be in JSON format:

In [19]:
for ans in answer_embeddings:
    print(json_to_string(ans))
    print("---------------------------------------------------")

"Total Revenue": [
{
"period": "12 Months Ended Dec. 31, 2023",
"value": 8971,
"sheet": "NOW-US, IS FY'23",
"cell": "C14"
},
{
"period": "12 Months Ended Dec. 31, 2022",
"value": 7245,
"sheet": "NOW-US, IS FY'23",
"cell": "D14"
},
{
"period": "12 Months Ended Dec. 31, 2021",
"value": 5896,
"sheet": "NOW-US, IS FY'23",
"cell": "E14"
}
]
---------------------------------------------------


You can see how the filtered table looks like. Window size was set to 1, so the most similar cells and their neighors 
are in the filtered excel.

Then this table was given to the llm.

<img src="images/sample_original.png" alt="filtered table" style="width: 600px;"/>
<img src="images/sample_filtered.png" alt="filtered table" style="width: 400px;"/>

The embeddings work great and scale well for large tables. However, for small tables, if you put the whole context to llm, 
it will do great as well:

In [20]:
def process_user_query_full_table(user_query, table_path):
    queries_formatted, filtered_query = get_formatted_quiries(user_query)
    embed_quiries(queries_formatted)

    query_answers_list = []

    answer = AnswerCall("")
    answer.load_table(table_path)

    for query in filtered_query.queries:
        print(f'{query =}')
        answer.query =  UserMessage(query)
        answer.respond()
        query_answers_list.append(answer.highlighted_json)
       
    return query_answers_list

Let's use the same query:

In [21]:
user_query = "Revenue"

In [22]:
answer_full_table = process_user_query_full_table(user_query, filepath) 

filtered_query.queries = ['Total Revenue']
query ='Total Revenue'


In [23]:
for ans in answer_full_table:
    print(json_to_string(ans))
    print("---------------------------------------------------")

"Total Revenue": [
{
"period": "12 Months Ended Dec. 31, 2023",
"value": 8971,
"sheet": "NOW-US, IS FY'23",
"cell": "B14"
},
{
"period": "12 Months Ended Dec. 31, 2022",
"value": 7245,
"sheet": "NOW-US, IS FY'23",
"cell": "C14"
},
{
"period": "12 Months Ended Dec. 31, 2021",
"value": 5896,
"sheet": "NOW-US, IS FY'23",
"cell": "D14"
}
]
---------------------------------------------------


Let's try another query:

In [24]:
user_query = "Give me other other income and expenses"

In [25]:
answer = process_user_query_full_table(user_query, filepath) 

filtered_query.queries = ['Other Income', 'Other Expenses']
query ='Other Income'
query ='Other Expenses'


In [26]:
for ans in answer:
    print(json_to_string(ans))
    print("---------------------------------------------------")

"Other Income": [
{
"period": "12 Months Ended Dec. 31, 2023",
"value": -56,
"sheet": "NOW-US, IS FY'23",
"cell": "C28"
},
{
"period": "12 Months Ended Dec. 31, 2022",
"value": -38,
"sheet": "NOW-US, IS FY'23",
"cell": "D28"
},
{
"period": "12 Months Ended Dec. 31, 2021",
"value": -28,
"sheet": "NOW-US, IS FY'23",
"cell": "E28"
}
]
---------------------------------------------------
"Other Expenses": [
{
"period": "12 Months Ended Dec. 31, 2023",
"value": -56,
"sheet": "NOW-US, IS FY'23",
"cell": "C28"
},
{
"period": "12 Months Ended Dec. 31, 2022",
"value": -38,
"sheet": "NOW-US, IS FY'23",
"cell": "D28"
},
{
"period": "12 Months Ended Dec. 31, 2021",
"value": -28,
"sheet": "NOW-US, IS FY'23",
"cell": "E28"
}
]
---------------------------------------------------


<img src="images/sample_original_other.png" alt="filtered table" style="width: 600px;"/>


Now let's try big cap table!

In [26]:
big_table_filepath = 'tutorial_data/complex_sheet.xlsx'
generate_embeddings(big_table_filepath)

In [27]:
user_query = "Revenue"

In [28]:
answer_big_table = process_user_query(user_query, big_table_filepath) 

filtered_query.queries = ['Total Revenue']
filtered excel for tutorial_data/complex_sheet.xlsx is being created at filtered_table.xlsx
291


In [29]:
for ans in answer_big_table:
    print(json_to_string(ans))
    print("---------------------------------------------------")

"Total Revenue": [
{
"period": "2021-12-31",
"value": 5750.5021,
"sheet": "IS",
"cell": "I16"
},
{
"period": "2022-12-31",
"value": 6250.94676,
"sheet": "IS",
"cell": "J16"
},
{
"period": "2023-12-31",
"value": 8251.11151666667,
"sheet": "IS",
"cell": "K16"
},
{
"period": "2024-12-31",
"value": 10298.9533333333,
"sheet": "IS",
"cell": "L16"
},
{
"period": "2025-12-31",
"value": 15220.02,
"sheet": "IS",
"cell": "M16"
},
{
"period": "2026-12-31",
"value": 21190.02,
"sheet": "IS",
"cell": "N16"
}
]
---------------------------------------------------


You can see how embeddings and filtering left only cells relevant to the income and their neighbors within
some window.

Next question is how to represent this extracted data to feed llm. Although CSV works better than the original table,
it still uses a lot of space for empty cells.

<img src="images/complex_original.png" alt="filtered table" style="width: 800px;"/>
<img src="images/complex_filtered.png" alt="filtered table" style="width: 800px;"/>
